In [14]:
import numpy as np
from sklearn.svm import SVC
from pyswarm import pso
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import MiniBatchSparsePCA
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
import pandas as pd
import pickle
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import random 
import os 
from sklearn.cross_decomposition import PLSRegression
from utils_AE import LRScheduler
import umap
from sklearn.metrics import roc_curve, auc
from collections import Counter

In [15]:
# Load features

path = "/root/workdir/IMMUCan/Files_TMA/Features_AE_raw/Milan_Hist_rawFeaturesConvAE10_10_64_split80-20.pickle"
        
#path = "/root/workdir/IMMUCan/Files_TMA/Features_AE_raw/Milan_Hist_rawFeatures60CropsConvAE_z3_3_64_split80-20.pickle"

with open(path,"rb",) as archivo:
    features = pickle.load(archivo)

if "data" not in features.keys():
    folds = {"data":{"fold0":features}}

In [16]:
classifier = "SVM" #"SVM" "RF" "MLP"
augmentation = True
components  = 10
reduction = "PLS" # "PCA" "PLS"  "LDA" "umap"
rcrop = None #60 

if classifier == "MLP":
    lr=0.01
    epochs = 200
    LrScheduler = True

In [17]:
def seed_everything(seed=228):
    """Function used to manage same seed and therefore reproducibility"""
    print(f"=>[REPLICABLE] True, with seed {seed}")
    print("    =>WARNING: SLOWER THIS WAY")
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

if classifier == "MLP":
    seed_everything()

In [18]:
if augmentation:
    print("SMOTE for Data Augmentation")
    
if classifier == "RF":
    #clf = RandomForestClassifier(n_estimators=200, max_depth=5, max_leaf_nodes=5, random_state=0)
    clf = RandomForestClassifier(n_estimators=10, random_state=0)
elif classifier == "SVM":
    kernel = 1.0 * RBF(1.0)    
    clf = make_pipeline(StandardScaler(), SVC(gamma=1e1, C=1e1, kernel=kernel))
    

if classifier == "MLP":    
    criterion = nn.CrossEntropyLoss()    
    class MLP(nn.Module):
        def __init__(self, input_size,  output_size):
            super(MLP, self).__init__()
            self.fc1 = nn.Linear(input_size, int(input_size*0.5))  
            self.bn1 = nn.BatchNorm1d(int(input_size*0.5))
            self.relu = nn.ReLU()  
            self.fc2 = nn.Linear(int(input_size*0.5), int(input_size*0.25))  
            self.bn2 = nn.BatchNorm1d(int(input_size*0.25))
            self.fc3 = nn.Linear(int(input_size*0.25), output_size)            
            self.dropout = nn.Dropout(p=0.3)

        def forward(self, x):
            x = self.fc1(x)   
            x = self.bn1(x)         
            x = self.relu(x)
            x = self.dropout(x)
            x = self.fc2(x)
            x = self.bn2(x)
            x = self.relu(x)
            x = self.dropout(x)
            x = self.fc3(x)
            x = torch.sigmoid(x)
            return x
      
    
##### Train model
scores = []
for f in folds["data"]:
    print("\n", f, "\n")
    X_train = folds["data"][f]["train"]["train_features"]
    X_val = folds["data"][f]["val"]["val_features"]
    val_names = folds["data"][f]["val"]['val_names']
    print("train set: ",X_train.shape)
    print("val set: ",X_val.shape)

        
    y_train = folds["data"][f]["train"]["train_y"]
    y_val = folds["data"][f]["val"]["val_y"]
    val_names = features["val"]["val_names"]

    ## augment data

    if augmentation:
        sm = BorderlineSMOTE(sampling_strategy='all',k_neighbors=5,random_state = 42)
        X_train, y_train = sm.fit_resample(X_train, y_train)

    ## dimensionality reduction
    if reduction == "PCA":
        pca = PCA(n_components=components)
        pca.fit(X_train)
        X_train = pca.fit_transform(X_train)
        X_val = pca.fit_transform(X_val)
    elif reduction == "LDA":
        print("reduction with LDA")
        components = 1
        lda = LinearDiscriminantAnalysis(n_components=components)
        X_train = lda.fit_transform(X_train, y_train)
        X_val = lda.transform(X_val)
        print("train: " ,X_train.shape)
        print("val: " ,X_val.shape)
    elif reduction == "PLS":
        print("reduction with PLS")
        pls = PLSRegression(n_components=components)
        pls.fit(X_train,y_train)
        X_train = pls.transform(X_train)
        X_val = pls.transform(X_val)
    elif reduction == "umap":
        print("reduction with UMAP")
        umap_model  = umap.UMAP(n_neighbors=10,min_dist=0.5,)
        X_train = umap_model.fit_transform(X_train)
        X_val = umap_model.transform(X_val)
           
    else:
        components=X_train.shape[1]

    print("new shape train: ",X_train.shape)
    print("new shape val: ",X_val.shape)

    if classifier == "RF" or classifier == "SVM":
        print("Classifier: ",classifier)
        clf.fit(X_train, y_train)        
        y_pred = clf.predict(X_val)

        if rcrop:
            print("\n","Majority voting with", rcrop," crops","\n")
            y_pred = torch.tensor(y_pred)
            y_val = torch.tensor(y_val)
            n_pred = []
            n_lab = []
            n_name = []
            percent = []
            for bp in range(int(len(y_pred) / rcrop)):
                ## majority voting
                n_pred.append(
                    torch.mode(
                        y_pred[
                            bp * rcrop : (bp + 1) * rcrop
                        ].squeeze()
                    ).values.item()
                )
                ### percentage
                per0 = (sum(y_pred[
                            bp * rcrop : (bp + 1) * rcrop
                        ].squeeze())*100)/rcrop
                per1 = (rcrop - sum(y_pred[
                            bp * rcrop : (bp + 1) * rcrop
                        ].squeeze()))*100 /rcrop
                if per0>per1:
                    percent.append(per0.item())
                else:
                    percent.append(per1.item())
               
                """# If at least 1 crop predicted as 1, the gloabl patient labels is 1
                n_pred.append(
                    int(
                        torch.any(
                            binary_predictions[
                                bp * n_crops : (bp + 1) * n_crops
                            ].squeeze()
                            == 1
                        )
                    )
                )"""
                n_lab.append(
                    y_val[bp * rcrop : (bp + 1) * rcrop][0].item()
                )
                n_name.append(val_names[bp * rcrop : (bp + 1) * rcrop][0])

            y_val = n_lab
            y_pred = n_pred

        if rcrop:
            correct = (
                sum(np.array(y_pred)==np.array(y_val))
            )
            accuracy = correct / len(y_val)
            cm = confusion_matrix(y_val,y_pred)
            fpr, tpr, thresholds = roc_curve(y_val,y_pred)
        else:
            accuracy = clf.score(X_val, y_val)
            cm = confusion_matrix(y_val,clf.predict(X_val))
            fpr, tpr, thresholds = roc_curve(y_val,clf.predict(X_val))


        f1 = f1_score(y_val, y_pred)

        print("acc: ",accuracy)
        print("f1: ",f1)
        scores.append(f1)

        
        print(cm)
        sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
        print("Sensitivity : ", sensitivity)

        specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
        print("Specificity : ", specificity)

        
        roc_auc = auc(fpr, tpr)            
        print("AUC: ",roc_auc)

    if classifier == "MLP":
        clf = MLP(input_size=components, output_size=2)
        optimizer = optim.Adam(clf.parameters(), lr=lr)
        if LrScheduler:
            lr_scheduler = LRScheduler(optimizer,patience=5)
        
        X_train, y_train = torch.Tensor(X_train), torch.LongTensor(y_train)
        X_val, y_val = torch.Tensor(X_val), torch.LongTensor(y_val)

        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)

        
        batch_size = 32
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        acc = []
        lval = []
        for epoch in range(epochs):
            clf.train()
            total_loss = 0

            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = clf(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            average_loss = total_loss / len(train_dataset)
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {average_loss:.4f}')

    
            clf.eval()
            correct = 0
            total = 0

            with torch.no_grad():
                total_loss = 0
                for batch_X, batch_y in val_loader:
                    outputs = clf(batch_X)
                    loss = criterion(outputs, batch_y)
                    total_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total += batch_y.size(0)
                    correct += (predicted == batch_y).sum().item()

            accuracy = 100 * correct / total
            average_loss = total_loss / len(val_dataset)
            lval.append(average_loss)
            acc.append(accuracy)
            print(f'Accuracy on validation set: {accuracy:.2f}%')

            if LrScheduler:
                lr_scheduler(average_loss)

        plt.plot(range(len(acc)), acc, label='val accuracy')
        plt.title("val acc")
        plt.ylim(50,100)
        plt.show()
        plt.plot(range(len(lval)), lval, label='val loss')
        plt.title("val loss")
        plt.show()
        f1 = f1_score(y_val, predicted)        
        print("f1: ",f1)        

        cm = confusion_matrix(y_val,predicted)
        print(cm)
        sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
        print("Sensitivity : ", sensitivity)

        specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
        print("Specificity : ", specificity)

        fpr, tpr, thresholds = roc_curve(y_val,predicted)
        roc_auc = auc(fpr, tpr)            
        print("AUC: ",roc_auc)
        


SMOTE for Data Augmentation

 fold0 

train set:  (117, 6400)
val set:  (29, 6400)
reduction with PLS
new shape train:  (152, 10)
new shape val:  (29, 10)
Classifier:  SVM
acc:  0.896551724137931
f1:  0.7999999999999999
[[20  1]
 [ 2  6]]
Sensitivity :  0.9523809523809523
Specificity :  0.75
AUC:  0.8511904761904762
